# Введение в рекомендательные системы

## Коллаборативная фильтрация

В этом задании мы закончим имплементацию коллаборативной фильтрации.

Для этого - выполним действия, необходимые для создания матрицы рейтингов

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
filepath = './data/user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

In [ ]:
filepath = './data/movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')

# Перекодируем ID фильмов и пользователей

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
enc_user = LabelEncoder()
enc_mov = LabelEncoder()

In [ ]:
enc_user = enc_user.fit(df_rates.userID.values)
enc_mov = enc_mov.fit(df_rates.movieID.values)

In [ ]:
idx = df_movies.loc[:, 'id'].isin(df_rates.movieID)
df_movies = df_movies.loc[idx]

In [ ]:
df_rates.loc[:, 'userID'] = enc_user.transform(df_rates.loc[:, 'userID'].values)
df_rates.loc[:, 'movieID'] = enc_mov.transform(df_rates.loc[:, 'movieID'].values)
df_movies.loc[:, 'id'] = enc_mov.transform(df_movies.loc[:, 'id'].values)

In [ ]:
df_rates.head()

## Матрица рейтингов

In [ ]:
from scipy.sparse import coo_matrix, csr_matrix

In [ ]:
R = coo_matrix((df_rates.rating.values, (df_rates.userID.values, df_rates.movieID.values)))

In [ ]:
R

## Похожесть между пользователями

В дальнейшем нам будет удобнее работать с форматом `Compressed Sparse Row matrix`. К счастью переформатировать полученную нами матрицу можно одной командой:

In [ ]:
R = R.tocsr()

Теперь, например, рейтинги для первого пользователя можно достать так:

In [ ]:
user_1 = R[0]
user_1

Так как вы возможно не работали с разреженным форматом матриц, устроим небольшой ликбез.

Первым делом, надо понадобится вектор для другого пользователя:

In [ ]:
user_2 = R[1]
user_2

Мы можем сравнивать элементы с 0

In [ ]:
user_1_rated = (user_1 != 0)
user_1_rated

Можем их "индексировать"

In [ ]:
user_1[user_1_rated]

Можем считать количество ненулевых элементов

In [ ]:
user_1.nnz

Можем умножать 2 разреженных вектора поэлементно:

In [ ]:
(user_1).multiply(user_2)

и скалярно

In [ ]:
user_1.dot(user_2.T)[0, 0]

И превращать разреженную матрицу (вектор) в плотную

In [ ]:
user_1_dense = user_1.toarray()
user_1_dense

Этого ликбеза вам будет должно быть достаточно, чтобы реализовать функцию расчета похожести между парой пользователей $u$ и $v$:

$$ s_{uv} = \frac{\sum\limits_{i \in I_u\cap I_v} R_{ui} R_{vi}}{\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{ui}^2}}\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{vi}^2}}}$$

Давайте будем считать, что если количество фильмов которые пользователь $u$ и $v$ посмотрели вместе $<= 2$, то их косинусная мера равна 0.0


### _Решение_

In [ ]:
def cosine_similarity_pair_users(u, v):
    u_rated = (u != 0)
    v_rated = (v != 0)
    common_items = (u_rated).multiply(v_rated)
    if common_items.nnz > 2:
        scalar = u.dot(v.T)[0, 0]
        norm = np.linalg.norm(u[common_items]) * np.linalg.norm(v[common_items])
        return scalar / norm
    else:
        return 0.0

## Функция прогнозирования рейтинга

Реализуйте функцию, которая принимает на входе
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей (несмотря на то, что каждый пользователь - ближайший сосед самому себе, в расчетах он использоваться не должен)

и возвращает вектор с предсказанными рейтингами по всем товарам для этого пользователя

Для того, чтобы считать прогноз по рейтингу мы воспользуемся упрощенной формулой из лекции:

$$ \hat{R}_{ui} = \frac{\sum_{v \in N(u)} s_{uv}R_{vi}}{\sum_{v \in N(u)} \left| s_{uv}\right|} $$


### _Решение_

In [ ]:
def rate_items_user(u, R, n_neigbours=30):
    predictions = csr_matrix((1, R.shape[1]))
    cumsim = 0.0
    num_predictions = csr_matrix((1, R.shape[1]))
    s = np.array([cosine_similarity_pair_users(R[u], R[v]) for v in range(R.shape[0])])
    similar_users = np.argsort(s)[::-1]
    print(np.sort(s)[::-1])
    for v in similar_users[:n_neigbours]:
        if v == u:
            continue
        user_sim = s[v]
        predictions += user_sim * R[v]
        cumsim += np.abs(user_sim)
    predictions /= cumsim
    return predictions

В качестве ответа к этому заданию верните 5 идентификаторов фильмов с наивысшим предсказанным рейтингом для пользователя с id 19 (20-я строчка в матрице рейтингов).
* Для усреднения используйте 30 ближайших соседей
* Среди этих 5-и фильмов не должно быть ранее просмотренных фильмов

### _Решение_

In [ ]:
R_hat = rate_items_user(20, R, n_neigbours=30)

In [ ]:
rated_items = (R[20] == 0)
unseen_ratings = R_hat.multiply(rated_items)
unseen_ratings = unseen_ratings.toarray()[0]

In [ ]:
idx = unseen_ratings.argsort()[::-1]

In [ ]:
unseen_ratings[idx]

In [ ]:
top5 = idx[:5]

In [ ]:
top5

In [ ]:
assert ', '.join(str(i) for i in top5) == '2614, 306, 343, 5573, 6720'

## Генерация ответа

In [ ]:
', '.join(str(i) for i in top5)